In [3]:
import mysql.connector
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='YOUR_PASSWORD',
    database='retail_sales'
)

# Write SQL query
query = """
SELECT 
    t.Store, t.Dept, t.Date, t.Weekly_Sales, t.IsHoliday,
    s.Type, s.Size,
    f.Temperature, f.Fuel_Price, f.CPI, f.Unemployment
FROM sales_train t
JOIN stores s ON t.Store = s.Store
JOIN features f ON t.Store = f.Store AND t.Date = f.Date;
"""

# Load into DataFrame
df = pd.read_sql(query, conn)

# Close connection
conn.close()


ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)

In [4]:
import mysql.connector
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='users'
)

# Write SQL query
query = """
SELECT 
    t.Store, t.Dept, t.Date, t.Weekly_Sales, t.IsHoliday,
    s.Type, s.Size,
    f.Temperature, f.Fuel_Price, f.CPI, f.Unemployment
FROM sales_train t
JOIN stores s ON t.Store = s.Store
JOIN features f ON t.Store = f.Store AND t.Date = f.Date;
"""

# Load into DataFrame
df = pd.read_sql(query, conn)

# Close connection
conn.close()


C:\Users\91942\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
# Convert date
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Fill missing values
df.fillna(method='ffill', inplace=True)


In [7]:
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week
df['Year'] = df['Date'].dt.year
df['IsHoliday'] = df['IsHoliday'].astype(str).str.strip().str.upper().map({'FALSE': 0, 'TRUE': 1})



In [8]:
features = ['Store', 'Dept', 'Month', 'Week', 'Year', 'IsHoliday',
            'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
X = df[features]
y = df['Weekly_Sales']



In [9]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = XGBRegressor()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)


RMSE: 6455.132919818414


In [10]:
results = X_test.copy()
results['Predicted_Sales'] = y_pred
results['Actual_Sales'] = y_test.values

# Save to CSV
results.to_csv("sales_predictions.csv", index=False)


In [11]:
print(df.columns)  # Make sure 'Date' is in there


Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Type', 'Size',
       'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Month', 'Week',
       'Year'],
      dtype='object')


In [12]:
results = X_test.copy()
results['Predicted_Sales'] = y_pred
results['Actual_Sales'] = y_test.values

# 🔍 Check if 'Date' is in results
print(results.columns)


Index(['Store', 'Dept', 'Month', 'Week', 'Year', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Predicted_Sales', 'Actual_Sales'],
      dtype='object')


In [13]:
if 'Date' not in results.columns and 'Date' in df.columns:
    X_test_with_date = X_test.copy()
    X_test_with_date['Date'] = df.loc[X_test.index, 'Date'].values
    results = X_test_with_date.copy()
    results['Predicted_Sales'] = y_pred
    results['Actual_Sales'] = y_test.values

# Save to CSV again
results.to_csv("sales_predictions.csv", index=False)
